<a href="https://colab.research.google.com/github/mgencler/qmem/blob/main/qbellek_temiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y qiskit qiskit-aer plotnine matplotlib

Found existing installation: qiskit 1.3.2
Uninstalling qiskit-1.3.2:
  Successfully uninstalled qiskit-1.3.2
Found existing installation: qiskit-aer 0.16.0
Uninstalling qiskit-aer-0.16.0:
  Successfully uninstalled qiskit-aer-0.16.0
Found existing installation: plotnine 0.14.5
Uninstalling plotnine-0.14.5:
  Successfully uninstalled plotnine-0.14.5
Found existing installation: matplotlib 3.7.1
Uninstalling matplotlib-3.7.1:
  Successfully uninstalled matplotlib-3.7.1


In [ ]:
!pip install -q "qiskit==1.3.2" "qiskit-aer==0.16.0" "matplotlib==3.8.0" "plotnine==0.14.5"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.9 MB/s eta 0:00:00


In [ ]:
# -*- coding: utf-8 -*-
"""
DÜZELTİLMİŞ KUANTUM BELLEK PROTOTİPİ (Tüm Durumlar Aktif)
"""

from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.circuit.library import UnitaryGate
import numpy as np

# 1. Özel Kapı Tanımları
class TopolojiKapilari:
    @staticmethod
    def homotopy(theta):
        return np.array([
            [np.cos(theta), -np.sin(theta), 0, 0],
            [np.sin(theta), np.cos(theta), 0, 0],
            [0, 0, np.cos(theta), np.sin(theta)],
            [0, 0, -np.sin(theta), np.cos(theta)]
        ])

    @staticmethod
    def F_operator(phi):
        return np.array([
            [np.cos(phi), 0, 0, 1j*np.sin(phi)],
            [0, np.cos(phi), -1j*np.sin(phi), 0],
            [0, -1j*np.sin(phi), np.cos(phi), 0],
            [1j*np.sin(phi), 0, 0, np.cos(phi)]
        ])

# 2. Kuantum Devresi
def create_circuit(theta=np.pi/2, phi=np.pi/2):
    qc = QuantumCircuit(2, 2)

    # Bell durumu oluştur
    qc.h(0)
    qc.cx(0, 1)

    # F operatörü
    qc.append(UnitaryGate(TopolojiKapilari.F_operator(phi)), [0,1])

    # Homotopy kapısı
    qc.append(UnitaryGate(TopolojiKapilari.homotopy(theta)), [0,1])

    # Ölçüm
    qc.measure([0,1], [0,1])

    return qc

# 3. Simülasyon
simulator = Aer.get_backend('aer_simulator')
qc = create_circuit(theta=np.pi/2, phi=np.pi/2)
qc_opt = transpile(qc, simulator, optimization_level=0)  # Optimizasyon kapalı
result = simulator.run(qc_opt, shots=8192).result()
counts = result.get_counts()

# 4. Sonuçlar
print("Ölçüm Sonuçları:", counts)
probs = np.array([counts.get(b, 0)/8192 for b in ['00','01','10','11']])
print("Olasılıklar:", probs)

Ölçüm Sonuçları: {'01': 4050, '10': 4142}
Olasılıklar: [0.         0.49438477 0.50561523 0.        ]
